# Exercise - Neural Network

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 



# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Ratio of delinquent to total number of credit lines
- Ratio of loan to value of current property
- Convert yr_renovated to a binary variable (i.e., renovated or not)
- (etc.)

In [25]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['ratio_loan_property_val'] = (df1['LOAN']/df1['VALUE']).fillna(0)
    df1['ratio_loan_property_val'].replace(np.inf, 1, inplace=True)
    

    return df1[['ratio_loan_property_val']]

In [26]:
new_col(train_inputs)

/var/folders/21/c170l60x7_v9gyz646wx98nr0000gn/T/ipykernel_49062/3068747483.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['ratio_loan_property_val'].replace(np.inf, 1, inplace=True)


,ratio_loan_property_val
1552,0.289149
2290,0.175701
1398,0.051985
1775,0.216491
2299,0.165515
...,...
1638,0.081112
1095,0.198824
1130,0.297348
1294,0.072863


##  Identify the numeric, binary, and categorical columns

In [27]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [28]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [29]:
categorical_columns

['REASON', 'JOB']

In [30]:
# define your feature engineered column(s) here
feat_eng_columns = ['LOAN','VALUE']

# Pipeline

In [31]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [32]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [33]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col))])



In [34]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
    
       ('trans', my_new_column, feat_eng_columns),
    
        ],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [35]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

/var/folders/21/c170l60x7_v9gyz646wx98nr0000gn/T/ipykernel_49062/3068747483.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['ratio_loan_property_val'].replace(np.inf, 1, inplace=True)


array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.28914899],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  0.        ,
         0.        ,  0.17570054],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.05198501],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.29734848],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.07286324],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.16781609]])

In [36]:
train_x.shape

(1750, 21)

# Tranform: transform() for TEST

In [37]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

/var/folders/21/c170l60x7_v9gyz646wx98nr0000gn/T/ipykernel_49062/3068747483.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['ratio_loan_property_val'].replace(np.inf, 1, inplace=True)


array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.16074572],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  1.        ,
         0.        ,  0.17635686],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.13547841],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  0.09874105],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.23121387],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.12917115]])

In [38]:
test_x.shape

(750, 21)

# Calculate the Baseline

In [39]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6034285714285714


In [42]:
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5773333333333334


# Train a shallow (one-layer) NN model

In [43]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(hidden_layer_sizes=(100,))

mlp_clf.fit(train_x, train_target)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

### Calculate the accuracy

In [45]:
from sklearn.metrics import accuracy_score



In [46]:
#Predict the train values
train_y_pred = mlp_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)



0.9108571428571428

In [47]:
#Predict the test values
test_y_pred = mlp_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.836

In [48]:
from sklearn.metrics import confusion_matrix

#confusion matrix on test set
confusion_matrix(test_target, test_y_pred)

array([[373,  60],
       [ 63, 254]])

# Train a deep (multi-layered) NN model 

In [49]:

dnn_clf = MLPClassifier(hidden_layer_sizes=(50,25,10),
                       max_iter=1000)

dnn_clf.fit(train_x, train_target)


MLPClassifier(hidden_layer_sizes=(50, 25, 10), max_iter=1000)

In [50]:

dnn_clf.n_iter_


329

In [53]:
dnn_clf.n_layers_

5

### Calculate the accuracy

In [56]:
#Predict the train values
train_y_pred = dnn_clf.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

1.0

In [57]:
#Predict the test values
test_y_pred = dnn_clf.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.844

# Optional: try grid search

In [65]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'hidden_layer_sizes': randint(low=5, high=100), 
              'max_iter': randint(low=100, high=1000)}

mlp_gs = RandomizedSearchCV(MLPClassifier(), param_grid, 
                             n_iter=15, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

mlp_gs.fit(train_x, train_target)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (471) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (471) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (471) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (471) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_

RandomizedSearchCV(cv=5, estimator=MLPClassifier(), n_iter=15,
                   param_distributions={'hidden_layer_sizes': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x136708bf0>,
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x136714a10>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [66]:
cvres = mlp_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8102857142857143 {'hidden_layer_sizes': 47, 'max_iter': 471}
0.7851428571428571 {'hidden_layer_sizes': 14, 'max_iter': 676}
0.8445714285714286 {'hidden_layer_sizes': 83, 'max_iter': 719}
0.7702857142857144 {'hidden_layer_sizes': 8, 'max_iter': 931}
0.7845714285714285 {'hidden_layer_sizes': 13, 'max_iter': 454}
0.8188571428571428 {'hidden_layer_sizes': 41, 'max_iter': 833}
0.78 {'hidden_layer_sizes': 32, 'max_iter': 261}
0.8337142857142856 {'hidden_layer_sizes': 50, 'max_iter': 992}
0.7725714285714286 {'hidden_layer_sizes': 15, 'max_iter': 626}
0.8451428571428572 {'hidden_layer_sizes': 83, 'max_iter': 960}
0.769142857142857 {'hidden_layer_sizes': 11, 'max_iter': 550}
0.7925714285714285 {'hidden_layer_sizes': 24, 'max_iter': 561}
0.8251428571428571 {'hidden_layer_sizes': 45, 'max_iter': 711}
0.836 {'hidden_layer_sizes': 84, 'max_iter': 475}
0.7651428571428572 {'hidden_layer_sizes': 8, 'max_iter': 342}


In [67]:
#Find the best parameter set
mlp_gs.best_params_

{'hidden_layer_sizes': 83, 'max_iter': 960}

In [68]:
mlp_gs.best_estimator_

MLPClassifier(hidden_layer_sizes=83, max_iter=960)

In [69]:
#Train accuracy:
train_y_pred = mlp_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

0.9948571428571429


In [70]:
#Test accuracy:
test_y_pred = mlp_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.864
